In [23]:
import pandas as pd
import geopandas as gpd
import numpy as np

import osmium
import re

In [24]:
# Qualifiers
def building_qualifier(tags):
   return ( ('building' in tags) or ('building:part' in tags) or (tags.get('type') == 'building') ) and ( (tags.get('location') != 'underground') or ('bridge' not in tags) )


class AreaHandler(osmium.SimpleHandler):

    def __init__(self):
        osmium.SimpleHandler.__init__(self)
        self.id = []
        self.type = []
        self.geometry = []
        self.wkbfab = osmium.geom.WKBFactory()
        
    def get_gdf(self):
        geometry = gpd.GeoSeries.from_wkb(self.geometry, crs='epsg:4326')
        id = pd.Series(self.id, dtype='UInt64')
        type = pd.Series(self.type, dtype='string')

        return gpd.GeoDataFrame({
            'id': id,
            'geometry': geometry,
            'type':  type
        }, index=geometry.index)
    
    def area(self, a):
        tags = dict(a.tags)
        id = int(a.orig_id())
        
        qualifier = building_qualifier(tags) # Use callback function to choose which qualifer: building qualifier or road network or poi/amenity!!
        if(qualifier):
            try:
                poly = self.wkbfab.create_multipolygon(a)     
                self.geometry.append(poly)
                self.id.append(id)
                if (not a.from_way()): self.type.append('R')
                else: self.type.append('W')
                
            except Exception as e:
                print(e)
                print(a)

In [25]:
h_r = AreaHandler()
h_r.apply_file("data/osm/rec.osm.pbf")

In [26]:
h_r.get_gdf()

,id,geometry,type
0,51816211,"MULTIPOLYGON (((-34.87171 -8.06429, -34.87155 ...",W
1,51816216,"MULTIPOLYGON (((-34.87398 -8.06525, -34.87389 ...",W
2,51816217,"MULTIPOLYGON (((-34.87413 -8.06455, -34.87409 ...",W
3,51816218,"MULTIPOLYGON (((-34.87267 -8.06534, -34.87264 ...",W
4,51816220,"MULTIPOLYGON (((-34.87218 -8.06516, -34.87202 ...",W
...,...,...,...
140736,1070775598,"MULTIPOLYGON (((-34.91519 -8.05162, -34.91519 ...",W
140737,1071164414,"MULTIPOLYGON (((-34.89924 -7.95401, -34.89923 ...",W
140738,1071828237,"MULTIPOLYGON (((-34.96853 -8.15485, -34.96819 ...",W
140739,13030020,"MULTIPOLYGON (((-34.89605 -8.08500, -34.89597 ...",R
